In [2]:
import numpy as np

def two_combinations(vector):
    """
    Convert an n-value vector to an n^2 2-dimensional vector representing all 2-combinations.

    :param vector: An n-value vector (list or numpy array).
    :return: An n^2 2-dimensional vector (2D numpy array).
    """
    n = len(vector)
    combination_vectors = []

    for i in range(n):
        for j in range(n):
            combination_vectors.append([vector[i], vector[j]])

    return np.array(combination_vectors)


In [3]:
import numpy as np
from scipy.stats import expon

def simulate_markov_batch(Q, initial_states, times, num_simulations):
    num_states = len(Q)
    num_times = len(times)
    states_at_times = np.zeros((num_simulations, num_times), dtype=int)

    for sim in range(num_simulations):
        state = initial_states
        states_at_times[sim, 0] = state

        for i in range(1, num_times):
            current_time = times[i - 1]
            end_time = times[i]
            while current_time < end_time:
                rate = -Q[state, state]
                time_to_next = expon.rvs(scale = 1/rate)
                #print(time_to_next)


                current_time += time_to_next
                if current_time < end_time:
                    transition_probs = Q[state, :] / rate
                    transition_probs[state] = 0
                    state = np.random.choice(num_states, p=transition_probs)

            states_at_times[sim, i] = state

    return states_at_times



def simulate_markov_batch_1(Q, initial_states, times, num_simulations , delta_t):
    num_states = len(Q)
    num_times = len(times)
    states_at_times = np.zeros((num_simulations, num_times), dtype=int)

    for sim in range(num_simulations):
        state = initial_states
        states_at_times[sim, 0] = state
        q = Q * delta_t
        I = np.identity(num_states)
        p = np.zeros((1,num_states))
        p[0,state] = 1
        for i in range(1, num_times):
          p = np.matmul(p,q+I)
          #print(p)
          state = np.random.choice(num_states, p=np.squeeze(p))
          states_at_times[sim, i] = state

    return states_at_times



In [120]:
expon.rvs( scale = 1/60)

0.0003763514443176389

In [28]:
np.sum(expon.rvs(size=18 , scale= 1/18) > 1)


0

In [4]:
import torch
import torch.nn.functional as F
from torch import nn, Tensor
import numpy as np
from torch.nn import Module, Linear, BatchNorm1d, Tanh
from numba import cuda
from torch import autograd
import torch.nn as nn
import torch.nn.functional as F
import math
"""
In this section we have used code available online in : https://github.com/frankhan91/DeepBSDE

"""

class StochasticProcess:
    """
    Base class for defining PDE related function.

    Args:
    eqn_config (dict): dictionary containing PDE configuration parameters

    Attributes:
    dim (int): dimensionality of the problem
    total_time (float): total time horizon
    num_time_interval (int): number of time steps
    delta_t (float): time step size
    sqrt_delta_t (float): square root of time step size
    y_init (None): initial value of the function
    """

    def __init__(self, eqn_config: dict):
        self.dim = eqn_config['dim']
        self.total_time = eqn_config['total_time']
        self.num_time_interval = eqn_config['num_time_interval']
        self.delta_t = self.total_time / self.num_time_interval
        self.sqrt_delta_t = np.sqrt(self.delta_t)
        self.y_init = None

    def sample(self, num_sample: int) -> Tensor:
        """
        Sample forward SDE.

        Args:
        num_sample (int): number of samples to generate

        Returns:
        Tensor: tensor of size [num_sample, dim+1] containing samples
        """
        raise NotImplementedError

    def r_u(self, t: float, x: Tensor, y: Tensor, z: Tensor) -> Tensor:
        """
        Interest rate in the PDE.

        Args:
        t (float): current time
        x (Tensor): tensor of size [batch_size, dim] containing space coordinates
        y (Tensor): tensor of size [batch_size, 1] containing function values
        z (Tensor): tensor of size [batch_size, dim] containing gradients

        Returns:
        Tensor: tensor of size [batch_size, 1] containing generator values
        """
        raise NotImplementedError

    def h_z(self, t,x,y,z: Tensor) -> Tensor:
        """
        Function to compute H(z) in the PDE.

        Args:
        h (float): value of H function
        z (Tensor): tensor of size [batch_size, dim] containing gradients

        Returns:
        Tensor: tensor of size [batch_size, dim] containing H(z)
        """
        raise NotImplementedError

    def terminal(self, x: Tensor) -> Tensor:
        """
        Terminal condition of the PDE.

        Args:
        t (float): current time
        x (Tensor): tensor of size [batch_size, dim] containing space coordinates

        Returns:
        Tensor: tensor of size [batch_size, 1] containing terminal values
        """
        raise NotImplementedError


class RFQ(StochasticProcess):
  """
  Args:
  eqn_config (dict): dictionary containing PDE configuration parameters
  """
  def __init__(self, basic_config,specific_config):
    super(RFQ, self).__init__(basic_config)

    self.n_liqiudity_state  = specific_config['nls']
    self.x_init = np.ones(self.dim) * specific_config['init']
    self.lamdas = specific_config['lamdas']
    self.lamda_initial_state = specific_config['init_state'] #integer
    self.sigma = specific_config['sigma']
    self.k = specific_config['k']
    self.Q = specific_config['Q']

  def sample(self, num_sample)-> tuple:
    """
    Sample forward SDE.

    Args:
    num_sample (int): number of samples to generate

    Returns:
    tuple: tuple of two tensors: dw_sample of size [num_sample, dim, num_time_interval] and
    x_sample of size [num_sample, dim, num_time_interval+1]
    """

    dw_sample = np.random.normal(size=[num_sample,  self.num_time_interval]) * self.sqrt_delta_t
    x_sample = np.zeros([num_sample, self.num_time_interval + 1])
    x_sample[:, 0] = np.ones(num_sample) * self.x_init

    select_Q = np.ones([num_sample, self.n_liqiudity_state**2  , self.n_liqiudity_state**2  ]) * np.expand_dims(np.exp(self.Q * self.delta_t), axis=0)
    select_lamda = np.ones([num_sample, self.n_liqiudity_state**2 , 2]) * np.expand_dims(two_combinations(self.lamdas), axis=0)
    #lamda_process = simulate_markov_batch(self.Q ,self.lamda_initial_state,np.array([i*self.delta_t for i in range(self.num_time_interval)]) ,num_sample )
    lamda_process = simulate_markov_batch_1(self.Q ,self.lamda_initial_state,np.array([i*self.delta_t for i in range(self.num_time_interval)]) ,num_sample ,  self.delta_t)

    ask_lamda = np.array([[self.lamdas[x // 2]for x in y] for y in  lamda_process ])
    bid_lamda = np.array([[self.lamdas[x % 2]for x in y] for y in  lamda_process ])

    for i in range(self.num_time_interval):

        x_sample[:,  i + 1] =  x_sample[:,  i ]+ (ask_lamda[:, i] -  bid_lamda[:, i]) * self.k * self.delta_t + self.sigma * dw_sample[:, i]
    return ask_lamda , bid_lamda, x_sample , lamda_process

  def r_u(self, t, x, y, z)-> torch.Tensor:
    """
    Interest rate in the PDE.

    Args:
    t (float): current time
    x (torch.Tensor): tensor of size [batch_size, dim] containing space coordinates
    y (torch.Tensor): tensor of size [batch_size, 1] containing function values
    z (torch.Tensor): tensor of size [batch_size, dim] containing gradients

    Returns:
    torch.Tensor: tensor of size [batch_size, 1] containing generator values
    """

    return 0

  def h_z(self,t,x,y,z)-> torch.Tensor:
      """
      Function to compute $h^T Z$ in the PDE.

      Args:
      t (float): current time
      x (torch.Tensor): tensor of size [batch_size, dim] containing space coordinates
      y (torch.Tensor): tensor of size [batch_size, 1] containing function value
      z (torch.Tensor): tensor of size [batch_size, dim] containing gradients

      Returns:
      torch.Tensor: tensor of size [batch_size, 1] containing H(z)
      """
      return 0


  def terminal(self,  x)-> torch.Tensor:
    """
    Terminal condition of the PDE.

    Args:
    t (float): current time
    x (torch.Tensor): tensor of size [batch_size, dim] containing space coordinates

    Returns:
    torch.Tensor: tensor of size [batch_size, 1] containing terminal values
    """
    return 0



In [5]:
"""A trading environment"""
import gym
from gym import spaces
from gym.utils import seeding

import numpy as np




class BoudEnv(gym.Env):
    """
    trading environment;
    """

    # trade_freq in unit of day, e.g 2: every 2 day; 0.5 twice a day;
    def __init__(self,basic_config,specific_config, num_sim=100):

        # simulated data: array of asset price, option price and delta paths (num_path x num_period)
        # generate data now
        self.sp = RFQ(basic_config,specific_config)



    def sample(self, batch_size):
      return  self.sp.sample(batch_size)




In [6]:
def weights_init_uniform_rule(m):
        classname = m.__class__.__name__
        # for every Linear layer in a model..
        if classname.find('nn.Linear') != -1:
            # get the number of the inputs
            n = m.in_features
            y = 1.0/np.sqrt(n)
            m.weight.data.uniform_(-y, y)
            m.bias.data.fill_(0)

class RL_Net(nn.Module):
    def __init__(self,INPUT_DIM,OUTPUT_DIM,HIDDEN_DIM , First = False):
        super(RL_Net, self).__init__()
        self.input_dim = INPUT_DIM
        self.output_dim = OUTPUT_DIM
        self.hidden_dim = HIDDEN_DIM
        self.is_first = First
        current_dim = self.input_dim
        self.layers = nn.ModuleList()
        self.bn = nn.ModuleList()
        self.droupout = nn.ModuleList() #drop out layer for regularization
        for hdim in self.hidden_dim:
            self.layers.append(nn.Linear(int(current_dim), int(hdim)))
            self.bn.append(nn.BatchNorm1d(int(hdim)))
            self.droupout.append(nn.Dropout(0.25)) # add a dropout layer
            current_dim = hdim
        self.layers.append(nn.Linear(int(current_dim), int(self.output_dim)))
    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            x = layer(x)
            if self.is_first == False:
              x = self.bn[i](x)
            x = F.tanh(x)
        out = self.layers[-1](x)
        return out

In [7]:
def scurve(delta , alpha , beta , delta_0):
  return 1/(1 + torch.exp(alpha + beta/delta_0 * delta))

In [8]:
def utility(ask_delta , bid_delta, q , z , ask_lamda, bid_lamda , gamma , alpha, beta, delta_0 , kappa , delta_t , sigma):
  t = ask_delta.shape[1]

  res = 0
  for i in range(t):
    res += (z*ask_delta[:,i] * ask_lamda[:,i]* scurve(ask_delta[:,i],alpha , beta, delta_0) + z*bid_delta[:,i] * bid_lamda[:,i]* scurve(bid_delta[:,i],alpha , beta, delta_0) + kappa*(ask_lamda[:,i] -bid_lamda[:,i] ) * q[:,i] - gamma/2 * q[:,i]**2 * sigma**2) * delta_t
  return torch.mean(res)


In [9]:
def fare_price(env,gamma,delta_0, z ,alpha, beta , batch_size = 200 , epoch= 100, device = torch.device('cpu')):
  ask_models = []
  bid_models = []
  i = 0
  n_model = env.sp.num_time_interval




  while i <= n_model-1:
      f= False

      model = RL_Net(4,1,[10,10,20,10]).to(device)



      model.apply(weights_init_uniform_rule)
      ask_models.append(model)

      model = RL_Net(4 ,1,[10,10,20,10]).to(device)



      model.apply(weights_init_uniform_rule)
      bid_models.append(model)
      i += 1




  optimizer_bid= torch.optim.Adam((par for model in bid_models
                    for par in model.parameters()),
                    lr=0.001, betas=(0.9, 0.99))
  optimizer_ask= torch.optim.Adam((par for model in ask_models
                    for par in model.parameters()),
                    lr=0.001, betas=(0.9, 0.99))




  for _ in range(epoch):
    ask_lamda, bid_lamda,prices,_ = env.sample(batch_size)
    ask_lamda = torch.tensor(ask_lamda).to(torch.float32)
    bid_lamda = torch.tensor(bid_lamda).to(torch.float32)
    prices = torch.tensor(prices).to(torch.float32)
    q = torch.zeros((batch_size ,env.sp.num_time_interval + 1))
    ask_delta = torch.zeros((batch_size ,env.sp.num_time_interval))
    bid_delta = torch.zeros((batch_size ,env.sp.num_time_interval))
    optimizer_ask.zero_grad()
    optimizer_bid.zero_grad()
    for t in range(env.sp.num_time_interval):

      q[:, t+1] = z*(bid_lamda[:,t] - ask_lamda[:,t])
      input = torch.cat((torch.ones((batch_size ,1 ))*t * env.sp.delta_t ,q[:, t:t+1]  , ask_lamda[:,t:t+1] , bid_lamda[:,t:t+1]), dim=1)
      ask_delta[:,t:t+1] = ask_models[t](input)
      bid_delta[:,t:t+1] = bid_models[t](input)

    loss = -1 * utility(ask_delta,bid_delta,q,z,ask_lamda, bid_lamda,gamma,alpha,beta,delta_0,env.sp.k,env.sp.delta_t,env.sp.sigma)
    print(-1 * loss)
    loss.backward()
    optimizer_ask.step()
    optimizer_bid.step()



In [10]:
Q = np.array([[-14.01, 4.37,4.37,5.27] , [19.32, -60.91,12.54,29.05] , [19.32,12.54,-60.91,29.05] , [23.67,15.00,15.00,-53.67]])

In [12]:
basic = {'dim' : 1 , 'total_time' : 0.25 , 'num_time_interval': 90 }
specific = {'init':103.593 , 'sigma':0.1839 ,  'nls' :2 , 'lamdas' : np.array([10.83, 73.03]) /10, 'init_state':1,'k' : 2.29 , 'Q': Q}

env = BoudEnv(basic , specific)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [164]:
env.sp.delta_t

0.002777777777777778

In [208]:
env.sample(1)[3]

array([[1, 1, 1, 2, 3, 1, 1, 3, 0, 2, 3, 2, 1, 0, 0, 0, 0, 0, 3, 1, 3, 2,
        0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 3,
        2, 3, 1, 0, 0, 0, 0, 3, 1, 0, 0, 3, 0, 1, 0, 3, 0, 0, 0, 2, 0, 0,
        0, 0, 3, 1, 2, 1, 0, 0, 3, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        3, 1]])

In [214]:
np.mean(np.sum((env.sample(4000)[0] - env.sample(4000)[1]) *env.sp.delta_t,axis = 1))

-0.10340318055555554

In [13]:
fare_price(env,0.0005 , 0.09,1,-0.7,3.1)

tensor(-0.8738, grad_fn=<MulBackward0>)
tensor(-0.8084, grad_fn=<MulBackward0>)
tensor(-0.7296, grad_fn=<MulBackward0>)
tensor(-0.6856, grad_fn=<MulBackward0>)
tensor(-0.6469, grad_fn=<MulBackward0>)
tensor(-0.6495, grad_fn=<MulBackward0>)
tensor(-0.7003, grad_fn=<MulBackward0>)
tensor(-0.6408, grad_fn=<MulBackward0>)
tensor(-0.6808, grad_fn=<MulBackward0>)
tensor(-0.5453, grad_fn=<MulBackward0>)
tensor(-0.5987, grad_fn=<MulBackward0>)
tensor(-0.5880, grad_fn=<MulBackward0>)
tensor(-0.6473, grad_fn=<MulBackward0>)
tensor(-0.5688, grad_fn=<MulBackward0>)
tensor(-0.5561, grad_fn=<MulBackward0>)
tensor(-0.6302, grad_fn=<MulBackward0>)
tensor(-0.5610, grad_fn=<MulBackward0>)
tensor(-0.5984, grad_fn=<MulBackward0>)
tensor(-0.6236, grad_fn=<MulBackward0>)
tensor(-0.5380, grad_fn=<MulBackward0>)
tensor(-0.6752, grad_fn=<MulBackward0>)
tensor(-0.5333, grad_fn=<MulBackward0>)
tensor(-0.5649, grad_fn=<MulBackward0>)
tensor(-0.6415, grad_fn=<MulBackward0>)
tensor(-0.5550, grad_fn=<MulBackward0>)
